In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tensorflow.keras import layers
from pathlib import Path
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.keras.layers import Dropout, Input, Add, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model

In [ ]:
path = Path("/content/drive/MyDrive/Face/face1")
filenames = list(map(lambda x: x.name, path.glob('*.jpg.chip.jpg')))

In [ ]:
print(filenames)
print(len(filenames))


['4_0_0_20170110213307248.jpg.chip.jpg', '4_0_3_20161220220754249.jpg.chip.jpg', '4_1_2_20161219142249665.jpg.chip.jpg', '4_0_2_20170110213515094.jpg.chip.jpg', '3_1_4_20161223231741788.jpg.chip.jpg', '4_1_4_20161221193328366.jpg.chip.jpg', '4_1_2_20161219221140063.jpg.chip.jpg', '4_0_2_20161219162919806.jpg.chip.jpg', '4_1_4_20161221202534346.jpg.chip.jpg', '4_0_0_20170110205414483.jpg.chip.jpg', '4_1_2_20161219192055923.jpg.chip.jpg', '4_1_2_20161219141750993.jpg.chip.jpg', '4_0_0_20170105183540567.jpg.chip.jpg', '4_0_0_20170110213530901.jpg.chip.jpg', '4_1_0_20170116200920314.jpg.chip.jpg', '4_1_0_20170116215618294.jpg.chip.jpg', '4_1_2_20161219142735921.jpg.chip.jpg', '4_1_4_20161221193352350.jpg.chip.jpg', '4_1_0_20170109193342601.jpg.chip.jpg', '4_1_1_20161219160501453.jpg.chip.jpg', '4_1_3_20161220220636202.jpg.chip.jpg', '4_0_3_20161220223111195.jpg.chip.jpg', '4_1_2_20161219163346526.jpg.chip.jpg', '4_1_0_20170109193119036.jpg.chip.jpg', '4_0_2_20161219200024691.jpg.chip.jpg',

In [ ]:
age_labels, gender_labels, image_path = [], [], []

for filename in filenames:
    image_path.append(filename)
    temp = filename.split('_')
    age_labels.append(temp[0])
    gender_labels.append(temp[1])

In [ ]:
df = pd.DataFrame()
df['image'], df['age'], df['gender'] = image_path, age_labels, gender_labels

In [ ]:
df['image'].describe

<bound method NDFrame.describe of 0         4_0_0_20170110213307248.jpg.chip.jpg
1         4_0_3_20161220220754249.jpg.chip.jpg
2         4_1_2_20161219142249665.jpg.chip.jpg
3         4_0_2_20170110213515094.jpg.chip.jpg
4         3_1_4_20161223231741788.jpg.chip.jpg
                         ...                  
16007    10_1_0_20170109205141310.jpg.chip.jpg
16008    10_0_2_20170110224230094.jpg.chip.jpg
16009    11_0_0_20170110225435539.jpg.chip.jpg
16010    10_1_3_20170109203848078.jpg.chip.jpg
16011    10_1_3_20170112211048149.jpg.chip.jpg
Name: image, Length: 16012, dtype: object>

In [ ]:
#print(max(df['age']))

In [ ]:
x = []
for i in range(len(df)):
    img = load_img("/content/drive/My Drive/Face/face1/"+df['image'].iloc[i])
    img =img.resize((128,128), Image.ANTIALIAS)
    ar = np.asarray(img)
    x.append(ar)


<ipython-input-8-be8dd3ece2d2>:4: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img =img.resize((128,128), Image.ANTIALIAS)


In [ ]:
x=np.array(x)

In [ ]:
y_age = np.array(df['age'])
y_age

array(['4', '4', '4', ..., '11', '10', '10'], dtype=object)

In [ ]:
y_gender = np.array(df['gender'])
y_gender

array(['0', '0', '1', ..., '0', '1', '1'], dtype=object)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y_age, test_size=0.2, random_state=42)

In [ ]:

X_gender_train, X_gender_test, y_gender_train, y_gender_test = train_test_split(
    x, y_gender, test_size=0.2, random_state=42)

In [ ]:
y_gender_train = y_gender_train.astype('float')
y_gender_test = y_gender_test.astype('float')

In [ ]:
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [ ]:

X_train.shape

(12809, 128, 128, 3)

In [ ]:
from keras.models import Sequential
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
# Define the CNN model for gender prediction
gender_model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128,128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification for gender
])


In [ ]:
gender_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
 #Train the age model
gender_model.fit(X_gender_train, y_gender_train, epochs=30, validation_data=(X_gender_test,y_gender_test),batch_size=32)


Epoch 1/30
401/401 [==============================] - 24s 29ms/step - loss: 1.9469 - accuracy: 0.7687 - val_loss: 0.3815 - val_accuracy: 0.8255
Epoch 2/30
401/401 [==============================] - 10s 24ms/step - loss: 0.3644 - accuracy: 0.8307 - val_loss: 0.3439 - val_accuracy: 0.8411
Epoch 3/30
401/401 [==============================] - 9s 23ms/step - loss: 0.3141 - accuracy: 0.8576 - val_loss: 0.3878 - val_accuracy: 0.8261
Epoch 4/30
401/401 [==============================] - 9s 24ms/step - loss: 0.2856 - accuracy: 0.8716 - val_loss: 0.3443 - val_accuracy: 0.8455
Epoch 5/30
401/401 [==============================] - 9s 23ms/step - loss: 0.2400 - accuracy: 0.8979 - val_loss: 0.3458 - val_accuracy: 0.8564
Epoch 6/30
401/401 [==============================] - 9s 24ms/step - loss: 0.2071 - accuracy: 0.9147 - val_loss: 0.3364 - val_accuracy: 0.8620
Epoch 7/30
401/401 [==============================] - 9s 23ms/step - loss: 0.1810 - accuracy: 0.9248 - val_loss: 0.3624 - val_accuracy: 0.86

In [ ]:
gender_model.save('Gender_prediction_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
resnet_base = ResNet50(include_top=False, weights='imagenet', input_shape=(128,128, 3))

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
# Freeze the convolutional layers in the base model
for layer in resnet_base.layers:
    layer.trainable = False

In [ ]:
model = Sequential([
    resnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='linear')  # Linear activation for regression
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError(), metrics=['mae'])


In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=200, validation_data=(X_test, y_test))

Epoch 1/200
401/401 [==============================] - 30s 62ms/step - loss: 62.8469 - mae: 5.4896 - val_loss: 49.7437 - val_mae: 5.3564
Epoch 2/200
401/401 [==============================] - 20s 51ms/step - loss: 45.5882 - mae: 4.8389 - val_loss: 37.2681 - val_mae: 4.5811
Epoch 3/200
401/401 [==============================] - 22s 54ms/step - loss: 38.6331 - mae: 4.5337 - val_loss: 34.1338 - val_mae: 4.3460
Epoch 4/200
401/401 [==============================] - 22s 55ms/step - loss: 35.4435 - mae: 4.3363 - val_loss: 34.8393 - val_mae: 4.4158
Epoch 5/200
401/401 [==============================] - 21s 51ms/step - loss: 32.3379 - mae: 4.1927 - val_loss: 39.4571 - val_mae: 4.8213
Epoch 6/200
401/401 [==============================] - 22s 54ms/step - loss: 30.7455 - mae: 4.1426 - val_loss: 32.6167 - val_mae: 4.2685
Epoch 7/200
401/401 [==============================] - 21s 51ms/step - loss: 28.4353 - mae: 3.9853 - val_loss: 32.9281 - val_mae: 4.2696
Epoch 8/200
401/401 [====================

In [ ]:
# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print("Test Mean Absolute Error:", test_mae)

101/101 [==============================] - 4s 41ms/step - loss: 41.0795 - mae: 4.7832
Test Mean Absolute Error: 4.783184051513672


In [ ]:
test_loss, test_accuracy = gender_model.evaluate(X_gender_test, y_gender_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

101/101 [==============================] - 1s 10ms/step - loss: 1.0231 - accuracy: 0.8530
Test Loss: 1.0230717658996582
Test Accuracy: 0.8529503345489502


In [ ]:
model.save('Age_prediction_model.h5')

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

# Load and preprocess the input image
image_path = "/content/sk.jpg"
image = load_img(image_path, target_size=(128,128))
  # Set target_size according to your model's input shape
img = img_to_array(image)
img = np.expand_dims(img, axis=0)  # Add a batch dimension
#img = preprocess_input(img)  # Preprocess the image
import matplotlib.pyplot as plt

# Display the image
plt.figure(figsize=(3,3))
plt.imshow(image)
plt.axis('off')  # Turn off axis labels
plt.show()


# Make the age prediction
age_prediction = model.predict(img)
Gender = gender_model.predict(img)
print("Predicted Age:", int(age_prediction[0][0]))  # age_prediction is a 2D array, so we access the value at [0][0]
predicted_gender = "Male" if Gender < 0.5 else "Female"

print(f'Predicted gender: {predicted_gender}')



FileNotFoundError: ignored